In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

2024-03-11 15:00:51.120561: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-11 15:00:51.227715: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-11 15:00:51.641167: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-11 15:00:53.108303: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def calculate_moving_averages(df, window_sizes):
    for window_size in window_sizes:
        column_name = f'MA_{window_size}'
        df[column_name] = df['close'].rolling(window=window_size).mean()
    return df

In [3]:
df = pd.read_csv('bnb_historical_data.csv')

In [4]:
window_sizes = [7, 25, 99]
df = calculate_moving_averages(df, window_sizes)

In [5]:
features = df[['MA_7', 'MA_25', 'MA_99', 'open', 'high', 'low', 'close', 'volume']].dropna()
target = features['close'].values.reshape(-1, 1)
features = features.values

# Normalize features using Min-Max scaling
scaler = MinMaxScaler(feature_range=(0, 1))
features_scaled = scaler.fit_transform(features)

In [6]:
features_scaled

array([[2.87707317e-01, 2.87457497e-01, 2.88031848e-01, ...,
        2.94117647e-01, 2.91242363e-01, 1.37694443e-03],
       [2.87512195e-01, 2.87375233e-01, 2.87983214e-01, ...,
        2.94117647e-01, 2.91242363e-01, 8.64287955e-04],
       [2.87609756e-01, 2.87320390e-01, 2.87941528e-01, ...,
        2.94117647e-01, 2.91921249e-01, 5.89021604e-04],
       ...,
       [7.29463415e-01, 7.30640562e-01, 7.22424478e-01, ...,
        7.32251521e-01, 7.31839783e-01, 4.01286874e-03],
       [7.29365854e-01, 7.30613140e-01, 7.22598171e-01, ...,
        7.32927654e-01, 7.31839783e-01, 9.71408071e-03],
       [7.29951220e-01, 7.30503455e-01, 7.22778812e-01, ...,
        7.32927654e-01, 7.32518669e-01, 8.31035114e-03]])

In [7]:
def create_sequences(data, target, sequence_length):
    sequences = []
    labels = []
    for i in range(len(data) - sequence_length):
        seq = data[i:i + sequence_length]
        label = target[i + sequence_length]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

In [8]:
sequence_length = 20  # Adjust as needed
X, y = create_sequences(features_scaled, target, sequence_length)

In [9]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2)) 
model.add(LSTM(units=50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

2024-03-11 15:01:07.618274: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 15:01:07.618754: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/jack/codes/ML_testing/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
model.fit(X, y, epochs=50, batch_size=32, validation_split=0.1)

Epoch 1/50


2024-03-11 15:01:13.852970: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 302907520 exceeds 10% of free system memory.


14789/14791 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 2151.1926

2024-03-11 15:05:57.852267: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 33656960 exceeds 10% of free system memory.


14791/14791 ━━━━━━━━━━━━━━━━━━━━ 293s 20ms/step - loss: 2150.9744 - val_loss: 1.7163
Epoch 2/50
14791/14791 ━━━━━━━━━━━━━━━━━━━━ 283s 19ms/step - loss: 770.0971 - val_loss: 142.1256
Epoch 3/50
14791/14791 ━━━━━━━━━━━━━━━━━━━━ 300s 20ms/step - loss: 726.7056 - val_loss: 97.9895
Epoch 4/50
14791/14791 ━━━━━━━━━━━━━━━━━━━━ 297s 20ms/step - loss: 691.3041 - val_loss: 20.2299
Epoch 5/50
14791/14791 ━━━━━━━━━━━━━━━━━━━━ 296s 20ms/step - loss: 654.0389 - val_loss: 13.5678
Epoch 6/50
14791/14791 ━━━━━━━━━━━━━━━━━━━━ 295s 20ms/step - loss: 596.9206 - val_loss: 7.1385


In [12]:
import pickle

In [24]:
with open('trained_model.pkl', 'wb') as file:
    pickle.dump(model, file)

AttributeError: Can't pickle local object 'Layer._initializer_tracker.<locals>.<lambda>'

In [ ]:
with open('trained_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

: 

In [12]:
df_24 = pd.read_csv('bnb_historical_data_24.csv')

combined_data = pd.concat([df, df_24], axis=0, ignore_index=True)

In [13]:
window_sizes = [7, 25, 99]
combined_data = calculate_moving_averages(combined_data, window_sizes)

In [14]:
features_24 = combined_data[['MA_7', 'MA_25', 'MA_99', 'open', 'high', 'low', 'close', 'volume']].dropna()
target_24 = features_24['close'].values.reshape(-1, 1)
features_24 = features_24.values

scaler_24 = MinMaxScaler(feature_range=(0, 1))
features_scaled_24 = scaler.fit_transform(features_24)

In [15]:
sequence_length = 20  # Adjust as needed
X_24, y_24 = create_sequences(features_scaled_24, target_24, sequence_length)

In [36]:
loss_24 = model.evaluate(X_24, y_24)
print(f'Mean Squared Error on Test Set: {loss_24}')

19028/19028 ━━━━━━━━━━━━━━━━━━━━ 119s 6ms/step - loss: 603.9608
Mean Squared Error on Test Set: 524.0720825195312


In [16]:
combined_data['predictions'] = model.predict(X_24)

In [ ]:
desired_months = [1, 2, 3]  # January, February, and March
filtered_df = df[df['timestamp'].dt.month.isin(desired_months)]

In [40]:
import plotly.graph_objects as go

candlestick_trace  = go.Candlestick(x=df['timestamp'],
                open=filtered_df['open'],
                high=filtered_df['high'],
                low=filtered_df['low'],
                close=filtered_df['close'])

symbol = 'BNBUSDT'

trace_ma_7 = go.Scatter(x=filtered_df['timestamp'], y=filtered_df['MA_7'], mode='lines', name='MA(7)', line=dict(color='green', dash='dash'))
trace_ma_25 = go.Scatter(x=filtered_df['timestamp'], y=filtered_df['MA_25'], mode='lines', name='MA(25)', line=dict(color='red', dash='dash'))
trace_ma_99 = go.Scatter(x=filtered_df['timestamp'], y=filtered_df['MA_99'], mode='lines', name='MA(99)', line=dict(color='purple', dash='dash'))

pred = go.Scatter(x=filtered_df['timestamp'], y=filtered_df['predictions'], mode='lines', name='predicted-price', line=dict(color='blue'))



fig = go.Figure(data=[candlestick_trace, trace_ma_7, trace_ma_25, trace_ma_99, pred])

fig.update_layout(title=f'Historical Price Data for {symbol}',
                  xaxis_title='Timestamp',
                  yaxis_title='Price (USDT)',
                  xaxis_rangeslider_visible=False)

# Show the interactive plot in a browser
fig.write_html('plot.html')